In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [ ]:
!pip install py7zr
import py7zr
import os

if not os.path.exists('/kaggle/train/') :
    os.makedirs('/kaggle/train/')


if not os.path.exists('/kaggle/test/') :
    os.makedirs('/kaggle/test/')



In [ ]:
with py7zr.SevenZipFile("/kaggle/input/statoil-iceberg-classifier-challenge/train.json.7z", 'r') as archive:
    archive.extractall(path="/kaggle/train")

with py7zr.SevenZipFile("/kaggle/input/statoil-iceberg-classifier-challenge/test.json.7z", 'r') as archive:
    archive.extractall(path="/kaggle/test")

In [ ]:
for dirname, _, filenames in os.walk('/kaggle'): 
    for filename in filenames: 
        print(os.path.join(dirname, filename))

In [ ]:
df_train = pd.read_json('/kaggle/train/data/processed/train.json')
df_test = pd.read_json('/kaggle/test/data/processed/test.json')

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import cv2
import keras

np.random.seed(1234)

def get_scaled_imgs(df):
    imgs = []

    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)

        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)
def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [ ]:
Xtrain = get_scaled_imgs(df_train)
Ytrain = np.array(df_train['is_iceberg'])

df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]
Xinc = df_train.inc_angle[idx_tr[0]]

Xtrain = get_more_images(Xtrain)
Xinc = np.concatenate((Xinc,Xinc,Xinc))
Ytrain = np.concatenate((Ytrain,Ytrain,Ytrain))

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
print(Xinc)

In [ ]:
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
Xinc = df_test.inc_angle
pred_test = model.predict([Xtest,Xinc])


In [ ]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': np.round(pred_test).astype('int16')[0,:, 0]})
print(submission.head(10))

submission.to_csv('cnn1.csv', index=False)

In [ ]:
pred_test = np.array(pred_test)

In [ ]:
pred_test.shape[0]

In [ ]:
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, AveragePooling2D, Flatten, Concatenate
from keras.optimizers import SGD
from keras.callbacks import Callback

from keras.utils import to_categorical
from keras.datasets import cifar10
import numpy as np

from keras.layers import Layer
from keras import backend as K
if K.backend() == 'theano':
    import theano.tensor as T
elif K.backend() == 'tensorflow':
    import tensorflow as tf
else:
    raise NotImplementedError
class LearningRateSchedule(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1)%8 == 0:
            lr = K.get_value(self.model.optimizer.lr)
            K.set_value(self.model.optimizer.lr, lr*0.96)

class LocalResponseNormalization(Layer):
    def __init__(self, n=5, alpha=1e-4, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x):
        _, r, c, f = self.shape 
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1,1), padding="same", pool_mode='avg')
        summed = K.sum(pooled, axis=3, keepdims=True)
        averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
            
        denom = K.pow(self.k + averaged, self.beta)
        
        return x / denom 
    
    def get_output_shape_for(self, input_shape):
        return input_shape

def Upscaling_Data(data_list, reshape_dim):
    ...

def inception(input_tensor, filter_channels):
    filter_1x1, filter_3x3_R, filter_3x3, filter_5x5_R, filter_5x5, pool_proj = filter_channels
    
    branch_1 = Conv2D(filter_1x1, (1, 1), strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(input_tensor)
    
    branch_2 = Conv2D(filter_3x3_R, (1, 1), strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(input_tensor)
    branch_2 = Conv2D(filter_3x3, (3, 3), strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(branch_2)

    branch_3 = Conv2D(filter_5x5_R, (1, 1), strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(input_tensor)
    branch_3 = Conv2D(filter_5x5, (5, 5), strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(branch_3)
    
    branch_4 = MaxPooling2D((3, 3), strides=1, padding='same')(input_tensor)
    branch_4 = Conv2D(pool_proj, (1, 1), strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(branch_4)
    
    DepthConcat = Concatenate()([branch_1, branch_2, branch_3, branch_4])
    
    return DepthConcat
    
def GoogLeNet(model_input, classes=1):
    conv_1 = Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(model_input) # (112, 112, 64)
    pool_1 = MaxPooling2D((3, 3), strides=2, padding='same')(conv_1) # (56, 56, 64)
    LRN_1 = LocalResponseNormalization()(pool_1) # (56, 56, 64)
    
    conv_2 = Conv2D(64, (1, 1), strides=1, padding='valid', activation='relu')(LRN_1) # (56, 56, 64)
    conv_3 = Conv2D(192, (3, 3), strides=1, padding='same', activation='relu')(conv_2) # (56, 56, 192)
    LRN_2 = LocalResponseNormalization()(conv_3) # (56, 56, 192)
    pool_2 = MaxPooling2D((3, 3), strides=2, padding='same')(LRN_2) # (28, 28, 192)
    
    inception_3a = inception(pool_2, [64, 96, 128, 16, 32, 32]) # (28, 28, 256)
    inception_3b = inception(inception_3a, [128, 128, 192, 32, 96, 64]) # (28, 28, 480)
    
    pool_3 = MaxPooling2D((3, 3), strides=2, padding='same')(inception_3b) # (14, 14, 480)
    
    inception_4a = inception(pool_3, [192, 96, 208, 16, 48, 64]) # (14, 14, 512)
    inception_4b = inception(inception_4a, [160, 112, 224, 24, 64, 64]) # (14, 14, 512)
    inception_4c = inception(inception_4b, [128, 128, 256, 24, 64, 64]) # (14, 14, 512)
    inception_4d = inception(inception_4c, [112, 144, 288, 32, 64, 64]) # (14, 14, 528)
    inception_4e = inception(inception_4d, [256, 160, 320, 32, 128, 128]) # (14, 14, 832)
    
    pool_4 = MaxPooling2D((3, 3), strides=2, padding='same')(inception_4e) # (7, 7, 832)
    
    inception_5a = inception(pool_4, [256, 160, 320, 32, 128, 128]) # (7, 7, 832)
    inception_5b = inception(inception_5a, [384, 192, 384, 48, 128, 128]) # (7, 7, 1024)
    
    avg_pool = GlobalAveragePooling2D()(inception_5b)
    dropout = Dropout(0.4)(avg_pool)
    
    linear = Dense(1, activation='relu')(dropout)
    
    model_output = Dense(classes, activation='sigmoid', name='main_classifier')(linear) # 'softmax'
    
    # Auxiliary Classifier
    auxiliary_4a = AveragePooling2D((5, 5), strides=3, padding='valid')(inception_4a)
    auxiliary_4a = Conv2D(128, (1, 1), strides=1, padding='same', activation='relu')(auxiliary_4a)
    auxiliary_4a = Flatten()(auxiliary_4a)
    auxiliary_4a = Dense(1024, activation='relu')(auxiliary_4a)
    auxiliary_4a = Dropout(0.7)(auxiliary_4a)
    auxiliary_4a = Dense(classes, activation='sigmoid', name='auxiliary_4a')(auxiliary_4a)
    
    auxiliary_4d = AveragePooling2D((5, 5), strides=3, padding='valid')(inception_4d)
    auxiliary_4d = Conv2D(128, (1, 1), strides=1, padding='same', activation='relu')(auxiliary_4d)
    auxiliary_4d = Flatten()(auxiliary_4d)
    auxiliary_4d = Dense(1024, activation='relu')(auxiliary_4d)
    auxiliary_4d = Dropout(0.7)(auxiliary_4d)
    auxiliary_4d = Dense(classes, activation='sigmoid', name='auxiliary_4d')(auxiliary_4d)
    
    
    model = Model(model_input, [model_output, auxiliary_4a, auxiliary_4d])
    
    return model


input_shape = (75, 75, 3)

model_input = Input( shape=input_shape )

model = GoogLeNet(model_input, 1)

optimizer = SGD(momentum=0.9)

model.compile(optimizer, 
        	loss={'main_classifier' : 'binary_crossentropy',
                    'auxiliary_4a' : 'binary_crossentropy',
                        'auxiliary_4d' : 'binary_crossentropy'},
                loss_weights={'main_classifier' : 1.0,
                                'auxiliary_4a' : 0.3, 
                                'auxiliary_4d' : 0.3}, 
                metrics=['acc'])

model.fit([Xtrain,Xinc],{'main_classifier' : Ytrain, 
                'auxiliary_4a' : Ytrain, 
                'auxiliary_4d' : Ytrain},  
        epochs=30, batch_size=24,verbose=1, callbacks=LearningRateSchedule())